# Introduction

***In progress.***

This notebook looks at building a simple RPG inventory system with Python. We can imagine this inventory system as an in-game backpack, suitcase, or other similar object that contains up to X number of items, based on how many "slots" (spaces which an item can fill) the player has available. Any item takes up X number of slots.

## Imports

In [10]:
import numpy as np

# `Inventory` Class

In [22]:
class Inventory:
    
    def __init__(self, total_slots = 16, items = []):
        self.total_slots = total_slots
        self.items = items
        self.used_slots = sum([x.slots for x in self.items])
        self.open_slots = self.total_slots - self.used_slots
        
    def __len__(self):
        return len(self.items)
        
    def update_slots(self):
        self.used_slots = sum([x.slots for x in self.items])
        self.open_slots = self.total_slots - self.used_slots 
        
    def add_item(self, item):
        # Check for Item class
        if isinstance(item, Item):
            if item.slots < self.open_slots: 
                self.items.append(item)
                # self.items.sort() # Can't sort Items, figure out how
                self.update_slots()
            else:
                print("Inventory Full")
        else:
            print("Not An Item")
            
    def remove_item(self, item):
        if item in self.items:
            self.items.remove(item)
        else:
            print("Item Not In Inventory")

# `Item` Class

***Needs description.***

In [16]:
class Item:

    def __init__(self, slots = 1):
        self.slots = slots

# Basic Examples

Example of adding items and summing inventory:

In [38]:
# Two items exist but have not been added to our Inventory

a = Item(2)
b = Item(3)

# See, no used slots

inv = Inventory()
inv.used_slots

0

In [39]:
# We have sixteen slots to fill

inv.open_slots

16

In [40]:
# We add two Items that take up five slots total and see that reflected

inv.add_item(a)
inv.add_item(b)

inv.used_slots

5

In [41]:
# Let's try and overload our Inventory with a big Item

c = Item(20)

inv.add_item(c)

Inventory Full


In [42]:
# Voila! We stopped it from overloading and stayed at the same slots

inv.used_slots

5

# `Item` Subclasses

***Need descriptions.***

> *Note to self: Investigate why `Potion` allows `slots = 1` and the `super()`, but `Ammo` throws an error with identical syntax.*

## `Potion`

In [83]:
class Potion(Item):
    def __init__(self, slots = 1, heal = False, effect = False):
        super().__init__(slots)
        self.heal = heal
        self.effect = effect
    
    def special_effect():
    # Gives player special effect
        pass
    
    def heal():
    # Heals player
        pass

## `Ammo`

`Ammo` for `Blaster` objects can be thought of as a magazine for a firearm, a cell of energy for a lazer blaster, or a similar source of projectiles/energy. When an object of type `Ammo` has its `has` become `0`, it should be destroyed.

In [84]:
class Ammo(Item):
    def __init__(self, slots, type_, holds = 8, has = 8):
        super().__init__(slots)
        self.type_ = type_
        self.holds = holds
        self.has = has
        self.slots = slots
        
    def consolidate_ammo(self, other_ammo):
    # Combine two mags of the same ammo
        pass

## `Weapon` And Subclasses

***Needs description.***

In [48]:
class Weapon(Item):
    def __init__(self, slots = 2, damage = 1, element = "normal",
                 special = False):
        super().__init__(slots)
        self.damage = damage
        self.element = element
        self.special = special
        
    def do_damage(self, target):
        if self.element in target.weak:
            target.hp_current -= (self.damage * 2)
        elif self.element in target.resist:
            target.hp_current -= (self.damage // 2)
        else:
            target.hp_current -= self.damage

In [49]:
class Blaster(Weapon):
    def __init__(self, slots = 2, damage = 1, element = "normal",
                 special = False, range_ = 75, mag = 8, ammo = "steel",
                 accuracy = 0.6):
        super().__init__(slots, damage, element, special)
        self.range_ = range_
        self.mag = mag
        self.mag_status = mag
        self.ammo = ammo
        self.accuracy = accuracy
        
    def shoot(self, target, distance):
        # Check if have ammo
        if self.mag_status > 0:
            # Check if target in range
            if distance < self.range_:
                # "Roll" to hit
                if np.random.random() < self.accuracy:
                    # Deal damage
                    self.do_damage(target)
                    # Decrement ammo
                    self.mag_status -= 1
                    print("Successful Hit")
                else:
                    self.mag_status -= 1
                    print("You Shot And Missed")
            else:
                print("Target Out Of Range")
        else:
            print("Out Of Ammo")
            
    def reload(self, ammo_source, inventory):
        # Check for object type
        if isinstance(ammo_source, Ammo):
            # Check for in-game type
            if ammo_source.type_ == self.ammo:
                # Add ammo based on capacity of blaster
                difference = self.mag - self.mag_status
                # If have more ammo than need
                if ammo_source.has > difference:
                    self.mag_status = self.mag
                    ammo_source.has -= difference
                    print(f"Reloaded - Ammo Has {ammo_source.has} Left")
                # If have less ammo than need
                elif ammo_source.has <= difference:
                    self.mag_status += ammo_source.has
                    inventory.remove_item(ammo_source)
                    print("Ammo Used Up")
            else:
                print("Wrong Ammo Type")
        else:
            print("That's Not Ammo")

## `Blaster.reload()` Example

In [111]:
# Create an Inventory

inv_0 = Inventory(total_slots = 8)

In [112]:
# Create Ammo

steel_rounds = Ammo(slots = 1, type_ = "steel")

In [113]:
# Add Ammo to Inventory

inv_0.add_item(steel_rounds)
inv_0.items

In [114]:
blaster = Blaster()

In [115]:
blaster.mag_status = 2

In [116]:
blaster.reload(ammo_source = steel_rounds, inventory = inv_0)

Reloaded - Ammo Has 2 Left


In [117]:
steel_rounds.has

2

# `Creature` Class

In [50]:
class Creature():
    def __init__(self, hp, resist = [], weak = []):
        self.hp_max = hp
        self.hp_current = hp
        self.resist = resist
        self.weak = weak

## `Creature` And `Blaster` Interactions

Example of `Creature` and `Blaster` class interacting.

In [56]:
flame_pistol = Blaster(element = 'fire', range_ = 6)
zombie = Creature(hp = 5, resist = ["poison"], weak = ["fire"])

In [58]:
flame_pistol.shoot(zombie, 3)

Successful Hit


In [59]:
# Notice the hp_current went down by 2, not 1, since it is weak to fire
zombie.hp_current

3

In [60]:
flame_pistol.mag_status

6

In [61]:
isinstance(flame_pistol, Blaster)

True